# SCRIPT TO OBTAIN PHENOTIPIC QUANTITATIVE TREATS

## This script should only be run once

#### Initialization
##### Load packages

In [ ]:
import re
import subprocess

import dxdata
import dxpy
import pyspark

from pyspark.sql import functions as F
from pyspark.conf import SparkConf
from pyspark.sql.types import StringType

from pathlib import Path
from phenotypes import get_pheno_fields, concatenate, new_names

Path("../tmp").resolve().mkdir(parents=True, exist_ok=True)

##### Spark and dataset configuration 

In [ ]:
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

dispensed_database_name = dxpy.find_one_data_object(
    classname="database", 
    name="app*", folder="/", name_mode="glob", 
    describe=True
)["describe"]["name"]
spark.sql("USE " + dispensed_database_name)

dispensed_dataset_id = dxpy.find_one_data_object(
    typename="Dataset", name="app*.dataset", folder="/", name_mode="glob"
)["id"]
dataset = dxdata.load_dataset(id=dispensed_dataset_id)

participant = dataset["participant"]

#### Data
##### Retrieve quantitative values of given fields

In [ ]:
age_sex_fields = get_pheno_fields(participant, fields=["31", "21022"])

quantitative_fields = get_pheno_fields(
    participant,
    fields=[
        "48",
        "49",
        "21001",
        "21002",
        "23099",
        "23127",
        "78",
        "30750",
        "30880",
        "26019",
        "4079",
        "4080",
    ],
)

field_names = concatenate(["eid"], age_sex_fields, quantitative_fields)

df = participant.retrieve_fields(names=field_names, engine=dxdata.connect())

##### DataFrame formatting

In [ ]:
# Drop arrays
to_drop = [x for x in df.columns if "a1" in x]
df = df.drop(*to_drop)

# Rename columns
colnames = [re.sub("_a\d", "", x) for x in df.columns]
colnames = ["xeid"] + [new_names(s) for s in colnames[1:]]

df = df.toDF(*colnames)

In [ ]:
# Print schema
df.printSchema()

##### Export and upload DataFrame

In [ ]:
df.write.csv("/tmp/phenos_QT.tsv", sep="\t", header=True, emptyValue="NA")

In [ ]:
!hadoop fs -getmerge /tmp/phenos_QT.tsv ../tmp/phenos_QT.tsv

!sed -e '3,${/^xeid/d' -e '}' ../tmp/phenos_QT.tsv > ../tmp/phenos.QT.raw.tsv
!dx upload ../tmp/phenos.QT.raw.tsv --path  /Data/phenotypes/ --brief